# Problem Statement


A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

#### Create a Spark Session

In [2]:
from pyspark.sql import SparkSession

In [3]:
sessn = SparkSession.builder.appName('KMeansProject').getOrCreate()

#### Get the data and show its stats

In [4]:
csv_data = sessn.read.csv(path='hack_data.csv',header=True,inferSchema=True)

In [5]:
csv_data.count()

334

In [11]:
csv_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [6]:
csv_data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [7]:
csv_data.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [10]:
csv_data['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used',
         'Servers_Corrupted','Pages_Corrupted','WPM_Typing_Speed'].show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|            70.8|
|                   20.0|            408.5|              0|             3.57|            8.0|           71.28|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+
o

In [13]:
#Check if there are any nulls in the dataset
csv_data.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [14]:
for c in csv_data.columns:
    print('The number of nulls in Feature ',c,' are :',csv_data.where(csv_data[c].isNull()).count())

The number of nulls in Feature  Session_Connection_Time  are : 0
The number of nulls in Feature  Bytes Transferred  are : 0
The number of nulls in Feature  Kali_Trace_Used  are : 0
The number of nulls in Feature  Servers_Corrupted  are : 0
The number of nulls in Feature  Pages_Corrupted  are : 0
The number of nulls in Feature  Location  are : 0
The number of nulls in Feature  WPM_Typing_Speed  are : 0


#### Convert csv Data into MLLib compatible format

In [12]:
from pyspark.ml.feature import VectorAssembler

In [15]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used',
                                       'Servers_Corrupted','Pages_Corrupted','WPM_Typing_Speed'],
                           outputCol='features')

In [16]:
vec_data = assembler.transform(csv_data)

In [17]:
vec_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
vec_data.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|[31.0,356.32,1.0,...|
|                    2.0|           228.08|              1|             2.48|     

In [19]:
vec_data.head(5)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37])),
 Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, Location='British Virgin Islands', WPM_Typing_Speed=69.08, features=DenseVector([20.0, 720.99, 0.0, 3.04, 9.0, 69.08])),
 Row(Session_Connection_Time=31.0, Bytes Transferred=356.32, Kali_Trace_Used=1, Servers_Corrupted=3.71, Pages_Corrupted=8.0, Location='Tokelau', WPM_Typing_Speed=70.58, features=DenseVector([31.0, 356.32, 1.0, 3.71, 8.0, 70.58])),
 Row(Session_Connection_Time=2.0, Bytes Transferred=228.08, Kali_Trace_Used=1, Servers_Corrupted=2.48, Pages_Corrupted=8.0, Location='Bolivia', WPM_Typing_Speed=70.8, features=DenseVector([2.0, 228.08, 1.0, 2.48, 8.0, 70.8])),
 Row(Session_Connection_Time=20.0, Bytes Transferred=408.5, Kali_T

In [20]:
vec_data.count()

334

#### Scale the Data

In [22]:
from pyspark.ml.feature import StandardScaler

In [24]:
scaler_ob = StandardScaler(inputCol='features',outputCol='scaledFeature')

In [25]:
scaled_data = scaler_ob.fit(vec_data).transform(vec_data)

In [26]:
scaled_data.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|       scaledFeature|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58

In [32]:
scaled_data.select('features','scaledFeature').show(5)

+--------------------+--------------------+
|            features|       scaledFeature|
+--------------------+--------------------+
|[8.0,391.09,1.0,2...|[0.56785108466505...|
|[20.0,720.99,0.0,...|[1.41962771166263...|
|[31.0,356.32,1.0,...|[2.20042295307707...|
|[2.0,228.08,1.0,2...|[0.14196277116626...|
|[20.0,408.5,0.0,3...|[1.41962771166263...|
+--------------------+--------------------+
only showing top 5 rows



In [33]:
scaled_data.head(5)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), scaledFeature=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963])),
 Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, Location='British Virgin Islands', WPM_Typing_Speed=69.08, features=DenseVector([20.0, 720.99, 0.0, 3.04, 9.0, 69.08]), scaledFeature=DenseVector([1.4196, 2.518, 0.0, 1.3206, 2.9378, 5.151])),
 Row(Session_Connection_Time=31.0, Bytes Transferred=356.32, Kali_Trace_Used=1, Servers_Corrupted=3.71, Pages_Corrupted=8.0, Location='Tokelau', WPM_Typing_Speed=70.58, features=DenseVector([31.0, 356.32, 1.0, 3.71, 8.0, 70.58]), scaledFeature=DenseVector([2.2004, 1.2444, 1.9976, 1.6117, 2.6114, 5.2628])),
 Row(Session_Connection_Time=2.0, Bytes Transferred=228.08, Kali_Tr

#### Building/Training KMeans Clustering Model with scaled data

In [34]:
from pyspark.ml.clustering import KMeans

##### Create 3 Cluster Model

In [56]:
kmeans3_ob = KMeans(featuresCol='scaledFeature',k=3)

In [57]:
kmeans3_model = kmeans3_ob.fit(scaled_data)

In [58]:
kmeans3_model.computeCost(scaled_data)

434.1492898715845

In [59]:
kmeans3_model.clusterCenters()

[array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
        3.28474   ]),
 array([1.30217042, 1.25830099, 0.        , 1.35793211, 2.57251009,
        5.24230473]),
 array([1.21780112, 1.37901802, 1.99757683, 1.37198977, 2.55237797,
        5.29152222])]

In [72]:
pred3 = kmeans3_model.transform(scaled_data)

In [73]:
pred3.select(['features','scaledFeature','prediction']).show(5)

+--------------------+--------------------+----------+
|            features|       scaledFeature|prediction|
+--------------------+--------------------+----------+
|[8.0,391.09,1.0,2...|[0.56785108466505...|         2|
|[20.0,720.99,0.0,...|[1.41962771166263...|         1|
|[31.0,356.32,1.0,...|[2.20042295307707...|         2|
|[2.0,228.08,1.0,2...|[0.14196277116626...|         2|
|[20.0,408.5,0.0,3...|[1.41962771166263...|         1|
+--------------------+--------------------+----------+
only showing top 5 rows



In [74]:
pred3.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+



In [76]:
pred3.groupBy('Kali_Trace_Used').count().show()

+---------------+-----+
|Kali_Trace_Used|count|
+---------------+-----+
|              1|  171|
|              0|  163|
+---------------+-----+



In [75]:
pred3.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'features',
 'scaledFeature',
 'prediction']

In [64]:
pred3.where(pred['prediction']==1).show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|       scaledFeature|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+----------+
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|         1|
|                   20.0|            408.5|              0|             3.57|            8.0|                Iraq|           71.28|[20.0,408.5,0.0,3...|[1.41962771166263...|         1|
|                   32.0|           242.48|              0|             4.2

###### Create 2 Cluster Model

In [68]:
kmeans2_ob = KMeans(featuresCol='scaledFeature',k=2)

In [69]:
kmeans2_model = kmeans2_ob.fit(scaled_data)

In [71]:
kmeans2_model.computeCost(scaled_data)

601.7707512676716

In [77]:
pred2 = kmeans2_model.transform(scaled_data)

In [80]:
pred2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [81]:
pred3.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+



###### It apears from the above results that there were exactly 2 Hackers since 3-Cluster didn't give the uniform distribution